In [ ]:
import pandas as pd
import numpy as np
import math
from tqdm import tqdm
import os
import csv
import json
import itertools
from os.path import join
from utils import get_repo_dir, datetime_str, logger, jsonify, sweep
from config.config import *
from typing import Union, Tuple
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import gc

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
from torch.nn import functional as F

## TODO
* Add yard line and orientation features

## Some initial variables and data loading

In [ ]:
max_num_frames = 30  # truncate if snap-to-throw is > this. units: ds
num_features = 11  # number of channels in input (vx, vy, etc)
num_extras = 1  # number of (play-level) features added in to outer network (just LoS for now)
out_dir = join(get_repo_dir(), 'output')
os.makedirs(out_dir, exist_ok=True)
exps_dir = join(out_dir, 'exps')
os.makedirs(exps_dir, exist_ok=True)

"""
Game Blacklist.
2018092301: Tony Jefferson data bad
"""
game_blacklist = [2018092301]

In [ ]:
full_data_path = join(data_dir, 'full_data.pkl')
if not os.path.isfile(full_data_path):
    weeks = range(1, 18)
    tracking_data = pd.concat([pd.read_csv(join(data_dir, f'week{week_num}_norm.csv')) for week_num in tqdm(weeks)], axis=0)
    coverage_data = pd.read_csv(join(data_dir, 'coverages_2018.csv')).dropna(subset=['coverage'])
    coverage_data.coverage.replace({'3 Seam': 'Cover 3 Zone', 'Cover 1 Double': 'Cover 1 Man'}, inplace=True)
    coverage_data = coverage_data[~coverage_data.coverage.isin(['Bracket', 'Mis'])]
    coverage_data.coverage = coverage_data.coverage.astype('category')
    full_data = pd.merge(tracking_data, coverage_data, how='inner', left_on=['gameId', 'playId'], right_on=['game_id', 'play_id'])
    full_data['coverage_code'] = full_data.coverage.cat.codes
    # save full data
    full_data.reset_index(drop=True).to_pickle(full_data_path)
    
    del tracking_data
    gc.collect()
else:
    # load full data from previously saved zip
    full_data = pd.read_pickle(full_data_path)
coverage_label_map = dict(zip(full_data['coverage'].cat.codes, full_data['coverage']))
num_classes = len(coverage_label_map)
coverage_label_map

TODO data aug. this is very slightly complicated by the fact that we have precomputed the vx, vy, ax, ay. will have to ask udit about how this is computed since it seems to be inconsistent with v_theta and v_mag?

for now we don't have orientation relative to qb, so I don't think it makes sense to highlight the qb in the way the bdb winner does for rusher, since it would just be distance from qb and qb-relative speed/accel data right now, which I doubt is _extra_ useful for predicting coverage (?). so for now, we'll do 11x11 of off x def with features:
* relative: x, y, vx, vy, ax, ay
* absolute: vx, vy, ax, ay

## Dataset generation

First, we generate the dataset as numpy arrays.

In [ ]:
def gen_data():
    grouped = full_data.groupby(['gameId', 'playId'])
    # TODO can change 11s to max number of off and def players in data
    data_X = np.empty((len(grouped), max_num_frames, num_features, 11, 11), dtype=np.float32)  # (P, T, F, D, O): play, frame, feature, def, off
    data_dims = np.empty((len(grouped), 3), dtype=np.int32) # (P, 3) contains (t, d, o): num frames, num def, num off on each play
    data_extras = np.empty((len(grouped), num_extras), dtype=np.float32)  # just LoS for now
    data_Y = np.empty(len(grouped), dtype=np.int32)  # (P,)
    
    valid_plays = 0
    for (game_id, play_id), play_df in tqdm(grouped):
        if game_id in game_blacklist:
            continue
        try:
            first_frame = play_df.loc[(play_df.nflId == 0) & (play_df.event == 'ball_snap')].frameId.iloc[0]
            play_end_frame = play_df.loc[(play_df.nflId == 0) & (play_df.event.isin(['pass_forward', 'pass_shovel', 'qb_sack', 'qb_strip_sack', 'tackle']))].frameId.iloc[0]
        except:
            print(f'({game_id}, {play_id}) failed. events were {play_df.event.unique()}')
            continue
        last_frame = min(first_frame + max_num_frames - 1, play_end_frame)
        play_df = play_df.loc[play_df.frameId.between(first_frame, last_frame)]

        num_def, num_off = 0, 0
        bad_play = False
        for frame_idx, (frame_id, frame_df) in enumerate(play_df.groupby('frameId')):
            def_ids = frame_df[frame_df.team_pos == 'DEF'].index
            num_def = len(def_ids)
            off_ids = frame_df[frame_df.team_pos == 'OFF'].index
            num_off = len(off_ids)
            qb_id = frame_df
            if num_def > 11 or num_off > 11:
                print(f'({game_id}, {play_id}), num_def {num_def}, num_off {num_off}')
                bad_play = True
                break
            if num_def < 3 or num_off < 3:
                print(f'({game_id}, {play_id}), num_def {num_def}, num_off {num_off}')
                bad_play = True
                break
            if (num_qbs := frame_df.loc[frame_df.position == 'QB'].shape[0]) != 1:
                print(f'({game_id}, {play_id}), num QBs {num_qbs}')
                bad_play = True
                break

            outer_sub = np.subtract.outer(
                frame_df.loc[off_ids, ['x', 'y', 'v_x', 'v_y', 'a_x', 'a_y']].values,
                frame_df.loc[def_ids, ['x', 'y', 'v_x', 'v_y', 'a_x', 'a_y']].values
            )
            if np.isnan(outer_sub).any():
                print(f'({game_id}, {play_id}), frame {frame_id} has NaNs')
                break
            # einsum explanation: the two i's get rid of subtraction across cols
            # k before j reorders def before off since output dims in alph. order
            data_X[valid_plays, frame_idx, :6, :num_def, :num_off] = np.einsum('kiji->ijk', outer_sub)
            data_X[valid_plays, frame_idx, 6:10, :num_def, :num_off] = frame_df.loc[def_ids, ['v_x', 'v_y', 'a_x', 'a_y']].values.T[...,None]
            data_X[valid_plays, frame_idx, 10, :num_def, :num_off] = np.mod(frame_df.loc[def_ids, 'o'].values[:,None] - frame_df.loc[frame_df.position == 'QB', 'o'].values, 360)
        if bad_play:
            continue
        data_dims[valid_plays] = last_frame - first_frame, num_def, num_off
        data_extras[valid_plays] = play_df.los.iloc[0]
        data_Y[valid_plays] = play_df.coverage_code.iloc[0]
        valid_plays += 1
    data_X = data_X[:valid_plays]
    data_dims = data_dims[:valid_plays]
    data_extras = data_extras[:valid_plays]
    data_Y = data_Y[:valid_plays]
    return data_X, data_dims, data_extras, data_Y

In [ ]:
data_save_path = join(out_dir, 'train_test_data.npz')
if not os.path.isfile(data_save_path):
    data_X, data_dims, data_extras, data_Y = gen_data()
    np.savez(data_save_path, x=data_X, dims=data_dims, extras=data_extras, y=data_Y)
else:
    saved_data = np.load(data_save_path)
    data_X, data_dims, data_extras, data_Y = saved_data['x'], saved_data['dims'], saved_data['extras'], saved_data['y']

Next, we make a [TensorDataset](https://pytorch.org/docs/stable/_modules/torch/utils/data/dataset.html#TensorDataset) out of it.

In [ ]:
train_X, test_X, train_dims, test_dims, train_extras, test_extras, train_Y, test_Y = train_test_split(
    data_X, data_dims, data_extras, data_Y, test_size=0.2
)
train_dataset = TensorDataset(*map(torch.tensor, [train_X, train_dims, train_extras, train_Y]))
test_dataset = TensorDataset(*map(torch.tensor, [test_X, test_dims, test_extras, test_Y]))

## Model construction

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True

In [ ]:
class DeepCoverInner(nn.Module):
    def __init__(self,
                 input_channels: int,
                 output_dim: int,
                 conv_h: Union[int, Tuple[int, int]]=[128, 96],
                 linear_h: Union[int, Tuple[int, int]]=[96, 256]):
        """
        :param input_channels: number of input features
        :param output_dim: dimension of output embedding
        :param conv_h: number of conv channels for each conv block.
            int or tuple of 2 ints.
        :param linear_h: number of hidden units for each linear layer.
            int or tuple of 2 ints.    
        """
        super().__init__()
        if type(conv_h) is int:
            conv_h = (conv_h, conv_h)
        if type(linear_h) is int:
            linear_h = (linear_h, linear_h)
        self.input_channels = input_channels
        self.output_dim = output_dim
        self.conv_h = conv_h
        self.linear_h = linear_h
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(input_channels, conv_h[0], kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(conv_h[0], conv_h[0], kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(conv_h[0], conv_h[0], kernel_size=1),
            nn.ReLU()
        )
        self.bn1 = nn.BatchNorm1d(conv_h[0])
        self.conv_block_2 = nn.Sequential(
            nn.Conv1d(conv_h[0], conv_h[1], kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm1d(conv_h[1]),
            nn.Conv1d(conv_h[1], conv_h[1], kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm1d(conv_h[1]),
            nn.Conv1d(conv_h[1], conv_h[1], kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm1d(conv_h[1]),
        )        
        self.linear_block = nn.Sequential(
            nn.Linear(conv_h[1], linear_h[0]),
            nn.ReLU(),
            nn.BatchNorm1d(linear_h[0]),
            nn.Linear(*linear_h),
            nn.ReLU(),
            nn.LayerNorm(linear_h[1]),
            nn.Linear(linear_h[1], output_dim)
        )
        
    def forward(self, x, dims):
        # let (F', F") and (..., F*) be conv_h and linear_h args to __init__
        orig_shape = x.shape  # (B, T, F, D, O)
        x = x.view(-1, *orig_shape[2:])  # (B*T, F, D, O)
        
        x = self.conv_block_1(x)  # (B*T, F', D, O)
        x = x.view(*orig_shape[:2], *x.shape[1:])  # (B, T, F', D, O)
        
        # this block of code handles variable number of offensive players                
        x_max = torch.stack([
            F.max_pool2d(each[...,:dim[2]], kernel_size=(1, dim[2])).squeeze() for each, dim in zip(x, dims)
        ])  # (B, T, F', D)
        x_avg = torch.stack([
            F.avg_pool2d(each[...,:dim[2]], kernel_size=(1, dim[2])).squeeze() for each, dim in zip(x, dims)
        ])  # (B, T, F', D)
        x = (x_max * 0.3 + x_avg * 0.7).view(-1, *x_max.shape[2:])  # (B*T, F', D)
        x = self.bn1(x)

        x = self.conv_block_2(x)
        x = x.view(*orig_shape[:2], *x.shape[1:])  # (B, T, F", D)
        # this block of code handles variable number of defensive players
        x_max = torch.stack([
            F.max_pool1d(each[...,:dim[1]], kernel_size=dim[1].item()).squeeze() for each, dim in zip(x, dims)
        ])  # (B, T, F")
        x_avg = torch.stack([
            F.avg_pool1d(each[...,:dim[1]], kernel_size=dim[1].item()).squeeze() for each, dim in zip(x, dims)
        ])  # (B, T, F")
        x = (x_max * 0.3 + x_avg * 0.7).view(-1, *x_max.shape[2:])  # (B*T, F")
        
        x = self.linear_block(x)  # (B*T, F*)
        
        # restore shape
        x = x.view(*orig_shape[:2], -1)  # (B, T, F*)
        return x


class DeepCoverOuter(nn.Module):
    def __init__(self,
                 input_dim: int,
                 output_dim: int,
                 hidden_size: int,
                 num_layers: int=1,
                 rnn_type: str='LSTM',
                 bidirectional: bool=True):
        """
        :param input_dim: input embedding dimension (per frame). same as Inner's output_dim
        :param output_dim: output embedding dimension
        :param hidden_size: dimension of LSTM hidden state
        :param num_layers: number of LSTM layers
        :param bidirectional: whether RNN is bidirectional
        """
        super().__init__()
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(input_dim, hidden_size, batch_first=True,
                               num_layers=num_layers, bidirectional=bidirectional)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(input_dim, hidden_size, batch_first=True,
                               num_layers=num_layers, bidirectional=bidirectional)
        else:
            raise ValueError(f'RNN type "{rnn_type}" not recognized.')
        self.num_layers = num_layers
        self.num_directions = 1 + int(bidirectional)
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.output_dim = output_dim
        self.input_dim = input_dim
        self.bidirectional = bidirectional
        self.linear = nn.Sequential(
            nn.Linear(self.num_layers * self.num_directions * hidden_size, output_dim),
            nn.ReLU()
        )
    
    def forward(self, x, dims):
        # x is (B, T, F*)
        batch_size = x.shape[0]
        x = nn.utils.rnn.pack_padded_sequence(x, dims[:,0].cpu(), batch_first=True, enforce_sorted=False)
        x = self.rnn(x)[1][0]  # last hidden state
        x = x.view(self.num_layers, self.num_directions, batch_size, -1)
        x = x.permute(2, 0, 1, 3).reshape(batch_size, -1)  # (B, F^)
        x = self.linear(x)
        return x
    

class DeepCover(nn.Module):
    def __init__(self,
                 inner_args: dict,
                 outer_args: dict,
                 num_classes: int,
                 num_extras: int
                ):
        """
        :param inner_args, outer_args: arguments to inner and outer networks
        :param num_classes: number of coverage classes to classify into
        :param num_extras: number of dimensions in extra input to classifier head
        """
        assert inner_args['output_dim'] == outer_args['input_dim']
        
        super().__init__()
        self.outer = DeepCoverOuter(**outer_args)
        self.inner = DeepCoverInner(**inner_args)
        self.num_classes = num_classes
        self.num_extras = num_extras
        self.classifier = nn.Sequential(
            nn.Linear(self.outer.output_dim + num_extras, num_classes),
            nn.LogSoftmax(dim=1)
        )
    
    def forward(self, x, dims, extras):
        x = self.inner(x, dims)
        x = self.outer(x, dims)
        x = self.classifier(torch.cat((x, extras), dim=1))
        return x

## Training

In [ ]:
def run_loop(model, dataloader, loss_fn, optimizer=None, train=True):
    if train:
        assert optimizer is not None, 'Optimizer must be specified in train mode.'
        model.train()
    else:
        model.eval()
    overall_loss = correct = 0
    for xb, dimb, exb, yb in dataloader:
        xb, dimb, exb, yb = xb.to(device), dimb.to(device), exb.to(device), yb.to(device)
        y_pred = model(xb, dimb, exb)
        loss = loss_fn(y_pred, yb.long())
        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        overall_loss += loss.item() * xb.shape[0]
        correct += (torch.max(y_pred, 1)[1] == yb).sum().item()
    acc = correct / len(dataloader.dataset)
    overall_loss /= len(dataloader.dataset)
    return acc, overall_loss

In [ ]:
def run_exp(params, exp_dir, log_to_console=True):
    """ Pre-exp variant check """
    model_params, training_params = params['model_params'], params['training_params']
    if model_params['num_layers'] == 1 and model_params['dropout'] != 0:
        # dropout makes no difference if there's only 1 layer so don't waste runs
        return
    models_dir = join(exp_dir, 'models')
    os.makedirs(models_dir)
    """ Logging setup """
    stats = {'train_loss': [], 'test_loss': [], 'train_acc': [], 'test_acc': []}
    with open(join(exp_dir, 'variant.json'), 'w') as f:
        jsonify(params, f)    
    f_log = open(join(exp_dir, 'train.log'), 'w')
    f_stats = open(join(exp_dir, 'stats.csv'), 'w')
    writer = csv.DictWriter(f_stats, fieldnames=['train_loss', 'train_acc', 'test_loss', 'test_acc'])
    writer.writeheader()
    
    """ Model setup """
    model = DeepCover(
        inner_args={
            'input_channels': model_params['num_features'],
            'output_dim': model_params['embedding_dim']
        },
        outer_args={
            'input_dim': model_params['embedding_dim'],
            'hidden_size': model_params['hidden_size'],
            'output_dim': model_params['outer_output_dim'],
            'bidirectional': model_params['bidirectional'],
            'rnn_type': model_params['rnn_type'],
            'num_layers': model_params['num_layers']
        },
        num_classes=num_classes,
        num_extras=num_extras
    ).to(device)
    
    """ Training setup """
    loss_fn = nn.NLLLoss()
    num_epochs = training_params['num_epochs']
    batch_size = training_params['batch_size']
    save_freq = training_params['save_freq']
    scheduler_type = training_params['scheduler_type']
    best_test_acc = 0
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
    optimizer = torch.optim.Adam(model.parameters(), lr=training_params['learning_rate'])
    if scheduler_type == 'exponential':
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.97)
    elif scheduler_type == 'plateau':
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=np.sqrt(0.1), patience=8, min_lr=9e-5)
    else:
        raise ValueError(f'Scheduler type "{scheduler_type}" not recognized.')

    try:
        for epoch in tqdm(range(num_epochs)):
            """ Training """
            train_acc, train_loss = run_loop(model, train_loader, loss_fn, optimizer, train=True)

            """ Evaluation"""
            test_acc, test_loss = run_loop(model, test_loader, loss_fn, train=False)

            scheduler.step(test_loss)
            """ Logging """
            stats['train_loss'].append(train_loss)
            stats['test_loss'].append(test_loss)
            stats['train_acc'].append(train_acc)
            stats['test_acc'].append(test_acc)
            logger(f'Epoch {epoch+1:>2}/{num_epochs} | TrLoss {train_loss:>8.5f} | TrAcc {100*train_acc:>5.2f} | TeLoss {test_loss:>8.5f} | TeAcc {100*test_acc:>5.2f}',
                  file=f_log, log_to_console=log_to_console)
            writer.writerow({'train_loss': train_loss, 'train_acc': 100*train_acc, 'test_loss': test_loss, 'test_acc': 100*test_acc})
            if (epoch >= save_freq and (best_test_acc := max(test_acc, best_test_acc)) == test_acc) or \
                (epoch+1) % save_freq == 0 or epoch == num_epochs - 1: 
                # new best after halfway, or save period, or last epoch
                filename = datetime_str() + '.pt'
                torch.save(model.state_dict(), join(models_dir, filename))
                logger(f'Model saved at {join(exp_dir, filename)}', file=f_log, log_to_console=log_to_console)
    except KeyboardInterrupt as e:
        if input('Do you want to save the model? [y/N]: ').lower()[0] == 'y':
            filename = datetime_str() + '.pt'
            torch.save(model.state_dict(), join(models_dir, filename))
            logger(f'Model saved at {join(exp_dir, filename)}', file=f_log, log_to_console=log_to_console)
        raise e
    finally:
        f_log.close()
        f_stats.close()
    return stats

In [ ]:
def run_sweep(model_space, training_space, verbose=True):
    """
    :param model_space: dict mapping model hyperparams to list of values to search over
    :param training_space: dict mapping training hyperparams to list of values to search over
    Launches experiments sweeping over possible combinations of hyperparameters, saving resulting
    experiment directories in the returned directory path.
    """
    save_dir = join(exps_dir, datetime_str())
    if verbose:
        print(f'Running sweep in {save_dir}...')
    model_combos = sweep(model_space)
    training_combos = sweep(training_space)
    num_exps = len(model_combos) * len(training_combos)
    for exp_id, (model_params, training_params) in enumerate(itertools.product(model_combos, training_combos)):
        if verbose:
            print(f'Launching experiment {exp_id+1}/{num_exps}...')
        params = {'model_params': model_params, 'training_params': training_params}
        exp_dir = join(save_dir, str(exp_id).zfill(4))
        run_exp(params, exp_dir, log_to_console=False)

In [ ]:
# run sweep over hyperparameters
model_space = dict(
    num_features=[num_features],
    num_classes=[num_classes],
    embedding_dim=[8, 32, 128],  # per frame
    outer_output_dim=[8, 32],
    hidden_size=[8, 32, 128],
    bidirectional=[True],
    rnn_type=['LSTM', 'GRU'],
    num_layers=[1, 3],
    dropout=[0]
)
training_space = dict(
    num_epochs=[60],
    batch_size=[64],
    save_freq=[8],
    learning_rate=[1e-3],
    scheduler_type=['plateau']
)
run_sweep(model_space, training_space)

## Analysis

In [ ]:
# # trained on week 1
# best_model_path = join(models_dir, '03032021_040817/03032021_042056.pt')
# best_model_path = join(models_dir, '03042021_070643/03042021_073225.pt')
# best_model_path = join(models_dir, '03032021_092502/03032021_100205.pt')
# # trained on weeks 1-4
# best_model_path = join(models_dir, '03042021_135230/03042021_155248.pt')
# best_model_path = join(models_dir, '03042021_173200/03042021_225324.pt')
# best_model_path = join(models_dir, '03052021_053603/03052021_145102.pt')
with open(join(os.path.dirname(best_model_path), 'variant.json'), 'r') as f:
    variant = json.load(f)
best_model = DeepCover(
    inner_args={
        'input_channels': num_features,
        'output_dim': variant['embedding_dim']
    },
    outer_type=variant.get('outer_type', 'LSTM'),  # back compat.
    outer_args={
        'input_dim': variant['embedding_dim'],
        'hidden_size': variant['hidden_size'],
        'num_classes': num_classes,
        'bidirectional': variant['bidirectional']
    }
).to(device)
best_model.load_state_dict(torch.load(best_model_path))
best_model.eval()
y_preds = []
ys = []
correct = 0
dataloader = test_loader
for xb, dimb, yb in dataloader:
    xb, dimb, yb = xb.to(device), dimb.to(device), yb.to(device)
    y_logits = best_model(xb, dimb)
    y_pred = torch.max(y_logits, 1)[1]
    y_preds.append(y_pred)
    ys.append(yb)
    correct += (y_pred == yb).sum()
acc = correct / len(dataloader.dataset)
print(f'Accuracy: {acc*100:.3f}%')
y_preds = torch.cat(y_preds).flatten()
ys = torch.cat(ys).flatten()
cm = confusion_matrix(ys, y_preds)
ConfusionMatrixDisplay(cm).plot()

In [ ]:
coverage_label_map